# Networks: structure, evolution & processes
**Internet Analytics - Lab 2**

---

**Group:** *Your group letter.*

**Names:**

* *Name 1*
* *Name 2*
* *Name 3*

---

#### Instructions

*This is a template for part 2 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [1]:
import requests
from random import randint

---

## 2.2 Network sampling

#### Exercise 2.7: Random walk on the Facebook network

In [5]:
# Base url of the API
URL_TEMPLATE = 'http://iccluster118.iccluster.epfl.ch:5050/v1.0/facebook?user={user_id}'

In [6]:
def naiveRandomWalkAge(steps, first_id):
    #Initial setup
    cid = first_id
    ageStack = 0
    
    for i in range(steps): 
        
        #Get current id information from server
        url = URL_TEMPLATE.format(user_id=cid)
        response = requests.get(url)
        data = response.json()
        
        #Add current age to stack
        ageStack += data['age']
        
        #Choose randomly next friend
        friends = data['friends']
        nextFriendIndex = randint(0, len(friends) - 1)
        cid = friends[nextFriendIndex]
    
    return (ageStack / steps)    

In [7]:
naiveRandomWalkAge(1000, 'f30ff3966f16ed62f5165a229a19b319')

25.52

From the previous algorithm, we can see that visiting 1000 nodes gave us an average age of 25.52

#### Exercise 2.8

1) Knowing that the average age on facebook is 43.3, we can see that our estimation is not accurate (more than 17 years difference!).

2) This can be explained by the fact that younger people might add others more easily on facebook. Thus their average degree is superior to the one of older people which explains why our random walk is completely biased. As seen in class, the random walk sees uniformly random edges which implies that nodes are biased by their number of edges. 

3) In order to have a better estimation, we can adjust our estimator to make it unbiased. This can be done using the formula seen in class when the total number of nodes and edges in the network is not known. 

In [8]:
def betterRandomWalkAge(steps, first_id):
    #Initial setup
    cid = first_id
    ageStack = 0
    denom = 0
    
    for i in range(steps): 
        
        #Get current id information from server
        url = URL_TEMPLATE.format(user_id=cid)
        response = requests.get(url)
        data = response.json()
        
        #Choose randomly next friend
        friends = data['friends']
        nextFriendIndex = randint(0, len(friends) - 1)
        cid = friends[nextFriendIndex]
        
        #Add current age to stack
        ageStack += data['age'] / len(friends)
        denom += 1 / len(friends)
    
    return (ageStack / denom)  

In [9]:
betterRandomWalkAge(1000, 'f30ff3966f16ed62f5165a229a19b319')

42.810953634965784

Now our estimator yields an average age of 42.8 while visiting the same number of nodes compare to the previous algorithm. This is quite close from reality.